# Imports

In [210]:
import pandas as pd # import for dataframe handle
import numpy as np # import for math and array operations
import matplotlib.pyplot as plt # import for visual representation
import seaborn as sns # import for visual representation

from bs4 import BeautifulSoup
import requests

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import string

# pipeline imports
from sklearn import set_config; set_config(display='diagram')
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# scalers, encoder, knn, vectorizer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import RobustScaler, StandardScaler, OneHotEncoder



%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [211]:
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ') # Remove Punctuation
        
    lowercased = text.lower() # Lower Case
    lemmatizer = WordNetLemmatizer()
    #lemmatized = [lemmatizer.lemmatize(word) for word in lowercased]
    #lowercased = lemmatized     
    tokenized = word_tokenize(lowercased) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('English')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words]# Remove Stop Words
    lemmatized = [lemmatizer.lemmatize(word) for word in without_stopwords]
    
    return " ".join(lemmatized)

# Load Dataset 

In [212]:
df = pd.read_csv('../raw_data/clean_df.csv' )
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata,cluster
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,multiplayer fast blood atmospheric zombie diff...,29
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,multiplayer based pvp access tactical adventur...,28
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,multiplayer based character mechs management r...,3
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,multiplayer indie open pvp access atmospheric ...,38
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,multiplayer mmorpg free open pvp capitalism at...,44


In [213]:
df.shape

(24564, 15)

In [214]:
df.isnull().sum()

url                     0
name                    0
developer              31
tags                    0
languages               0
genre                   3
game_description        0
mature_content          0
price                4555
reviews                 0
date                  256
achievements            0
op_sys              12380
metadata                0
cluster                 0
dtype: int64

In [215]:
df['tags'].fillna('', inplace=True)
df['genre'].fillna('', inplace=True)

In [216]:
df['metadata'] = df[['tags', 'genre']].apply(lambda x: ' '.join(x), axis = 1)


df['metadata'] = df['metadata'].apply(
    lambda x: ' '.join(list(set(x.split())))
)

In [217]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata,cluster
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,multiplayer fast blood atmospheric zombie diff...,29
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,multiplayer based pvp access tactical adventur...,28
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,multiplayer based character mechs management r...,3
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,multiplayer indie open pvp access atmospheric ...,38
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,multiplayer mmorpg free open pvp capitalism at...,44


In [218]:
df['metadata'] = df['metadata'].apply(clean_text)

df['game_description'] = df['game_description'].apply(clean_text)

In [219]:
df['tags'] = df['tags'].apply(clean_text)

In [220]:
df['languages'] = df['languages'].apply(clean_text)

In [221]:
df['genre'] = df['genre'].apply(clean_text)

In [222]:
df.head()

,url,name,developer,tags,languages,genre,game_description,mature_content,price,reviews,date,achievements,op_sys,metadata,cluster
0,https://store.steampowered.com/app/379720/DOOM/,DOOM,id Software,fps gore action demon shooter first person gre...,english french italian german spanish spain ja...,action,game developed id software studio pioneered fi...,0,17.991,Very Positive,2016-05-12 00:00:00,1.0,windows,multiplayer fast blood atmospheric zombie diff...,29
1,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,PUBG Corporation,survival shooter multiplayer battle royale pvp...,english korean simplified chinese french germa...,action adventure massively multiplayer,game playerunknown battleground battle royale ...,1,26.991,Mixed,2017-12-21 00:00:00,1.0,windows,multiplayer based pvp access tactical adventur...,28
2,https://store.steampowered.com/app/637090/BATT...,BATTLETECH,Harebrained Schemes,mechs strategy turn based turn based tactic sc...,english french german russian,action adventure strategy,game original battletech mechwarrior creator j...,0,35.991,Mostly Positive,2018-04-24 00:00:00,1.0,windows macOS Linux,multiplayer based character mechs management r...,3
3,https://store.steampowered.com/app/221100/DayZ/,DayZ,Bohemia Interactive,survival zombie open world multiplayer pvp mas...,english french italian german spanish spain cz...,action adventure massively multiplayer,game post soviet country chernarus struck unkn...,0,40.491,Mixed,2018-12-13 00:00:00,0.0,windows,multiplayer indie open pvp access atmospheric ...,38
4,https://store.steampowered.com/app/8500/EVE_On...,EVE Online,CCP,space massively multiplayer sci fi sandbox mmo...,english german russian french,action free play massively multiplayer rpg str...,game,0,NaN,Mostly Positive,2003-05-06 00:00:00,0.0,windows macOS,multiplayer mmorpg free open pvp capitalism at...,44


In [223]:
df.to_csv('../raw_data/clean_df.csv', index=False)

# Preprocessing

In [224]:
def kmeans_labels(df, n , mi):
    vec = TfidfVectorizer(min_df = mi ,ngram_range=(1,2))
    X = vec.fit_transform(df['game_description'])
    kmodel = KMeans(n_clusters=n)
    kmodel.fit(X)
    
    return kmodel.labels_

In [225]:
def create_pipeline(df, m=0.05 , c=1, n =50, mi = 0.04):
    array_transf = FunctionTransformer(lambda array: array.toarray())
    df['cluster'] = kmeans_labels(df, n, mi)
    
    
    meta_transf = make_pipeline(
        TfidfVectorizer(min_df=m), 
        array_transf,
        RobustScaler()
    )
    
    ord_encoder = OrdinalEncoder(
        categories=[
            [
                "Overwhelmingly Negative",
                "Very Negative",
                "Negative",
                "Mostly Negative",
                'Mixed',
                "Mostly Positive",
                "Positive",
                "Very Positive",
                "Overwhelmingly Positive"
            ]],
        dtype=np.int64,
        handle_unknown="use_encoded_value",
        unknown_value=-1
    )
    
    ord_transf = make_pipeline(
        ord_encoder, 
        StandardScaler())
    
    cluster_transf = make_pipeline(
        OneHotEncoder(sparse=False), 
         StandardScaler()
    )
    
    num_transf = make_pipeline(StandardScaler())


    preproc_basic = make_column_transformer(
        (meta_transf, 'metadata'),
        (cluster_transf, ['cluster']),
        (ord_transf, ['reviews']),
        (num_transf, ['mature_content', 'achievements']),
        remainder='drop'
    )
    
    full_pipe = make_pipeline(preproc_basic, PCA(n_components=c) )
    return full_pipe.fit_transform(df)
    
    #return preproc_basic.fit_transform(df)

In [226]:
def train(X, y):
    return KNeighborsRegressor().fit(X,y)

In [227]:
def recommending_system(model, X, game):
    
    neighbors_index = model.kneighbors(X.loc[[game]],n_neighbors=df.shape[0])[1][0]
    neighbors_distance = model.kneighbors(X.loc[[game]],n_neighbors=df.shape[0])[0][0]
    
    neighbors_list = list(neighbors_index)
    
    # new_df_values = {
    #     'distance': neighbors_distance,
    #     'url': [],
    #     'price': [],
    #     'reviews': [],
    #     'op_sys': [],
    #     'developer': [],
    # }
    
    # for index in neighbors_index:
    #     new_df_values['url'].append(df.loc[index, 'url'])
    #     new_df_values['price'].append(df.loc[index, 'price'])
    #     new_df_values['reviews'].append(df.loc[index, 'reviews'])
    #     new_df_values['op_sys'].append(df.loc[index, 'op_sys'])
    #     new_df_values['developer'].append(df.loc[index, 'developer'])
    
    return pd.DataFrame(neighbors_distance, index = X.iloc[neighbors_list, :].index, columns=['distance'])
    

In [228]:
pipe = create_pipeline(df)
pipe.shape

(24564, 1)

In [229]:
X = pd.DataFrame(pipe, index=df.name.tolist())

In [230]:
model = train(X, df['url'])

In [231]:
recommending_system(model, X, 'Portal 2').iloc[5211]

distance    0.480031
Name: Dungeon Girl, dtype: float64

# Testing Data

In [232]:
user_df = pd.read_csv('../raw_data/steam-200k.csv',usecols=[0,1,2,3],names=['userid','game','behavior','hoursplayed'])

In [233]:
# keeping only play entries

df_play = user_df[user_df['behavior']=='play']
df_play=df_play.drop(columns='behavior')

#keeping only games that are also in the main dataset
user_name= pd.DataFrame(df_play['game'].unique(),columns=['name']).merge(df, on = 'name')
join_name = list(user_name.name.unique())
df_play = df_play[df_play['game'].isin(join_name)]

In [234]:
# Creating DF of users favorites 2 games

def get_fav_games(df,user):
    db = df[df['userid']==user].sort_values(by='hoursplayed', ascending=False)
    return list(db['game'].iloc[0:2])

def get_user_list(df):
    temp_df=df.groupby('userid').count()[['game']]
    return list(temp_df[temp_df['game']>1].index)

def get_fav_list(df):
    user_list= get_user_list(df)
    fav_list=[]
    for user in user_list:
        fav_list.append(get_fav_games(df,user))
    fav1=[]
    fav2=[]
    for fav in fav_list:
        fav1.append(fav[0])
        fav2.append(fav[1])
    return pd.DataFrame(data=list(zip( fav1, fav2)),
                         columns=['most_fav_game', 'sec_fav_game'],index=user_list)


In [235]:
test_df = get_fav_list(df_play)

In [236]:
def testing_models(df, model):

    df['distance'] = ''
    for index, row in df.iterrows():
        neighbors_list = list(model.kneighbors(X.loc[[row['most_fav_game']]],n_neighbors=df.shape[0])[1][0])
        res = pd.DataFrame(model.kneighbors(X.loc[[row['most_fav_game']]],n_neighbors=df.shape[0])[0][0],\
                           index = X.iloc[neighbors_list, :]\
                        .index, columns = ['distance']).loc[row['sec_fav_game']][0]
        df.loc[index, 'distance'] = res
        
    return df

# Testing params

In [237]:
pipe = create_pipeline(df, 0.05, 1, 15, 0.2)
pipe.shape

(24564, 1)

In [238]:
X = pd.DataFrame(pipe, index=df.name.tolist())
model = train(X, df['url'])

In [239]:
recommending_system(model, X, 'Far Cry 3')

,distance
Far Cry 3,0.000000
Final Exam,0.000142
DEVIL GUNS,0.000243
Heart of Moon : The Mask of Seasons,0.000337
Combat Wings: Battle of Britain,0.000414
...,...
Curfew,3.949851
36 Fragments of Midnight,3.949851
Castle Of Cthulhu,3.949851
The Wanderings Dragon,3.949851


In [250]:
def test_params(test_df,df, X, model, name, index_name):
    for index, row in test_df.iterrows():
        neighbors_list = list(model.kneighbors(X.loc[[row['most_fav_game']]],n_neighbors=df.shape[0])[1][0])
        res = pd.DataFrame(model.kneighbors(X.loc[[row['most_fav_game']]],n_neighbors=df.shape[0])[0][0],\
                           index = X.iloc[neighbors_list, :]\
                        .index, columns = [name])
        test_df.loc[index, name] = res.loc[row['sec_fav_game']][0]
        test_df.loc[index, index_name]= res.index.get_loc(row['sec_fav_game'])
    return test_df

In [286]:
def check_params(df, m=0.05 , c=1, n =50, mi = 0.1):
    pipe = create_pipeline(df, m=0.05 , c=1, n =50, mi = 0.04)
    X = pd.DataFrame(pipe, index=df.name.tolist())
    model = train(X, df['url'])
    name = str(m)
    index_name = 'index' + str(m)
    test_params(test_df,df, X, model, name, index_name)
    return test_df

## Finding Best Params!

In [274]:
m_list = [0.03, 0.05, 0.7]
c_list = [1,5,10]
n_list = [10, 50, 70]
mi_list = [0.05, 0.07, 0.1]

In [275]:
main_list = [m_list, c_list, n_list, mi_list]

In [280]:
import itertools
params_list = list(itertools.product(*main_list))

In [291]:
# fun loop to go through!

for params in params_list:
    check_params(df, params)

KeyboardInterrupt: 